In [2]:
from pymongo import MongoClient
import pandas as pd
from bson.json_util import dumps
from bson.json_util import loads
import json

client = MongoClient('mongodb://localhost:27017/')
client.list_database_names()


['admin', 'config', 'local', 'raw_spotify', 'raw_spotify_test']

In [39]:
# %%time
db = client['raw_spotify']

aug_coll = db["raw_30-08-2021"]

cursor = list(aug_coll.find({'items' : { "$exists": True}}))

In [46]:
df = pd.DataFrame(cursor)

df= df.drop(['_id', 'next', 'cursors', 'limit', 'href'], axis=1)
df = df.explode('items', ignore_index=True).head(5)

df

,items
0,{'track': {'album': {'album_type': 'compilatio...
1,"{'track': {'album': {'album_type': 'album', 'a..."
2,"{'track': {'album': {'album_type': 'album', 'a..."
3,"{'track': {'album': {'album_type': 'album', 'a..."
4,"{'track': {'album': {'album_type': 'album', 'a..."


In [47]:
song_names = []
artist_names = []
played_at_list = []
timestamps = []

for record in df['items']:
    song_names.append(record["track"]["name"])
    artist_names.append(record["track"]["album"]["artists"][0]["name"])
    played_at_list.append(record["played_at"])
    timestamps.append(record["played_at"][0:10])
    

In [48]:
song_dict = {
    "song_name" : song_names,
    "artist_name": artist_names,
    "played_at" : played_at_list,
    "timestamp" : timestamps
}
song_df = pd.DataFrame(song_dict, columns = ["song_name", "artist_name", "played_at", "timestamp"])
song_df

,song_name,artist_name,played_at,timestamp
0,Still Loving You,Scorpions,2021-08-30T21:38:09.495Z,2021-08-30
1,Come and See Me (feat. Drake),PARTYNEXTDOOR,2021-08-30T21:36:31.353Z,2021-08-30
2,I Wish I Knew How It Would Feel to Be Free,Nina Simone,2021-08-30T21:31:27.995Z,2021-08-30
3,Baltimore,Nina Simone,2021-08-30T21:28:19.076Z,2021-08-30
4,I Put A Spell On You,Nina Simone,2021-08-30T21:23:37.150Z,2021-08-30
